## Analysis of Controller on Linear Simulation

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ..

In [ ]:
from __future__ import annotations

from glob import glob
import pickle

import matplotlib.pyplot as plt
import numpy as np

from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats('svg')

# hide top and right splines on plots
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False

In [ ]:
TIME_TICKS =  [   0, 2400, 4800,  7200,  9600, 12000, 14400]
TIME_LABELS = ['0h', '4h', '8h', '12h', '16h', '20h', '24h']

v_min, v_max = (11.4**2, 12.6**2)  # +/-5%, units kV^2
print(v_min, v_max)

y_min, y_max = 11.2, 12.8

In [ ]:
# Recreate Fig8 in Qu and Li (2020)
# - they count the substation as bus 1
# - we count the substation as bus -1
index = [9, 19, 22, 31, 40, 46, 55]  # for us, we want to plot nodes [7, 17, etc.]

In [ ]:
from network_utils import (
    create_56bus,
    create_RX_from_net,
    read_load_data)

net = create_56bus()
R, X = create_RX_from_net(net, noise=0)  # true R and X
p, qe = read_load_data()  # in MW and MVar
T, n = p.shape

v_nom = 12**2  # nominal squared voltage magnitude, units kV^2
v_sub = v_nom  # fixed squared voltage magnitude at substation, units kV^2

vpars = qe @ X + p @ R + v_sub  # shape [T, n]

In [ ]:
print('max-voltage node:', np.argmax(vpars.max(axis=0)))
print('min-voltage node:', np.argmin(vpars.min(axis=0)))

In [ ]:
# plot linear sim no-control
fig, ax = plt.subplots(figsize=(4, 3), dpi=200, tight_layout=True)

ts = range(T)
for i in np.asarray(index) - 2:
    ax.plot(ts, np.sqrt(vpars[:, i]))

ax.axhline(11.4, ls='--', color='black')
ax.axhline(12.6, ls='--', color='black')
ax.set(ylabel='Voltage (kV)', ylim=(y_min, 13.4))
ax.set(xlabel='time $t$', xlim=(0, T),
       xticks=TIME_TICKS, xticklabels=TIME_LABELS)

fig.savefig(f'plots/tsg/linear_nocontrol.pdf', bbox_inches='tight', dpi=200, pad_inches=0, facecolor='white')
fig.savefig(f'plots/tsg/linear_nocontrol.png', bbox_inches='tight', dpi=200, pad_inches=0, facecolor='white')

## Figures
1. voltage profile, robust controller with true X
2. voltage profile, robust controller+CBC with random X
3. voltage profile, robust controller+CBC with random X, known topo info for 14 buses
4. voltage profile, robust controller+CBC with random X, known topo+param info for 14 buses
5. error plot for $\|\hat{X}_t - X^*\|_\triangle$

In [ ]:
# use same seed for all cases
pkls_by_seed = {}
for seed in [8, 9, 10, 11]:
    pkls = {}
    pkl_paths = {
        'unknown': glob(f'out/CBCproj_noise1.0_perm_norm1.0_seed{seed}_2*.pkl')[0],
        'topo-14': glob(f'out/CBCproj_noise1.0_perm_norm1.0_seed{seed}_knowntopo14_2*.pkl')[0],
        'lines-14': glob(f'out/CBCproj_noise1.0_perm_norm1.0_seed{seed}_knownlines14_2*.pkl')[0],
        'known': 'out/CBCconst_20220211_052507.pkl',
    }

    for name, pkl_path in pkl_paths.items():
        with open(pkl_path, 'rb') as f:
            pkls[name] = pickle.load(f)
            print(pkls[name].keys())
    pkls_by_seed[seed] = pkls

In [ ]:
def plot_pkl(name: str, data: dict, plot_legend: bool = False) -> None:
    ts = range(T)
    fig, ax = plt.subplots(figsize=(4, 3), dpi=60, tight_layout=True)
    for i in np.array(index) - 2:
        ax.plot(ts, np.sqrt(data['vs'][:, i]), label=f'{i+1}')

    ax.axhline(11.4, ls='--', color='black')
    ax.axhline(12.6, ls='--', color='black')
    ax.set(ylabel='Voltage (kV)', ylim=(y_min, y_max),
           yticks=[11.4, 11.7, 12, 12.3, 12.6])
    ax.set(xlabel='time $t$', xlim=(-50, T),
           xticks=TIME_TICKS, xticklabels=TIME_LABELS)

    fig.savefig(f'plots/tsg/linear_{name}.pdf', bbox_inches='tight', dpi=200, pad_inches=0, facecolor='white')
    fig.savefig(f'plots/tsg/linear_{name}.png', bbox_inches='tight', dpi=200, pad_inches=0, facecolor='white')

    if plot_legend:
        leg = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), title='bus')
        fig.canvas.draw()
        bbox = leg.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
        fig.savefig('plots/tsg/linear_legend.pdf', dpi=200, bbox_inches=bbox, facecolor='white')
        fig.savefig('plots/tsg/linear_legend.png', dpi=200, bbox_inches=bbox, facecolor='white')

In [ ]:
seed = 10
for i, (name, data) in enumerate(pkls_by_seed[seed].items()):
    plot_pkl(name, data, plot_legend=(i == 0))

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=60, tight_layout=True)

for name, data in pkls.items():
    data = pkls[name]
    ax.plot(data['dists']['t'] + [T], data['dists']['true'] + [data['dists']['true'][-1]],
            label=name)
    ax.scatter(0, data['dists']['true'][0])
ax.legend()
ax.set_ylabel(r'$||\hat{X}_t - X^\star||_\bigtriangleup$')
ax.set(xticks=TIME_TICKS, xticklabels=TIME_LABELS)
ax.set(xlabel='time $t$', xlim=(-50, T))
fig.savefig(f'plots/tsg/linear_error.pdf', bbox_inches='tight', dpi=200, pad_inches=0, facecolor='white')
fig.savefig(f'plots/tsg/linear_error.png', bbox_inches='tight', dpi=200, pad_inches=0, facecolor='white')

## Figures (Alternative)

Only plot voltages for two buses: 18 and 30 (where bus 0 is substation)

In [ ]:
def plot_bus(bus: int, pkls: dict, plot_legend: bool = False) -> None:
    ts = range(T)
    fig, ax = plt.subplots(figsize=(4, 3), dpi=200, tight_layout=True)

    for name, data in pkls.items():
        ax.plot(ts, np.sqrt(data['vs'][:, bus-1]), label=name)

    ax.axhline(11.4, ls='--', color='black')
    ax.axhline(12.6, ls='--', color='black')
    ax.set(ylabel='Voltage (kV)', ylim=(y_min, y_max),
           yticks=[11.4, 11.7, 12, 12.3, 12.6])
    ax.set(xlabel='time $t$', xlim=(-50, T),
           xticks=TIME_TICKS, xticklabels=TIME_LABELS)

    fig.savefig(f'plots/tsg/linear_bus{bus}.pdf', bbox_inches='tight', dpi=200, pad_inches=0, facecolor='white')
    fig.savefig(f'plots/tsg/linear_bus{bus}.png', bbox_inches='tight', dpi=200, pad_inches=0, facecolor='white')

    if plot_legend:
        leg = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        fig.canvas.draw()
        bbox = leg.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
        fig.savefig('plots/tsg/linear_legend2.pdf', dpi=200, bbox_inches=bbox, facecolor='white')
        fig.savefig('plots/tsg/linear_legend2.png', dpi=200, bbox_inches=bbox, facecolor='white')

In [ ]:
for i, bus in enumerate([18, 30]):
    plot_bus(bus, pkls_by_seed[10], plot_legend=(i==0))